# ABSTRACT
Annotating a large-scale point cloud dataset for tasks like 3D object classification, segmentation, and detection can be time-consuming due to the irregular nature of point clouds. Self-supervised learning, which eliminates the need for manual labeling, presents a hopeful solution to this challenge. [1]
crosspoint_intro.png

# The method
The code aims to establish the relationship between 3D objects and 2D images in order to effectively learn transferable point cloud representations. We provide a reimplementation of the methods in the reference paper in PyTorch Lightning which is a high-level library for PyTorch and allows to easily build, train, and evaluate complex deep learning models. It provides a convenient and modular structure for PyTorch code, making it easier to scale, debug, and reuse code. 

The concept is to embeds augmented versions of the same point cloud close together in the feature space while maintaining their correspondence with the original 3D point cloud and its rendered 2D image; in our approach we have tested  other combinations of models to extract feature from sample.


# Our Work

In this project we suggest a reimplementation of CrossPoint, a straightforward cross-modal contrastive learning technique, to learn transferable 3D point cloud representations. It allows for a match between 3D objects and 2D images by maximizing consistency in the invariant space between point clouds and the related rendered 2D image, while encouraging invariance to transformations in the point cloud modality.

Being more precise we added to the original paper the Naive point Cloud transformer presented in Meng-Hao Guo et al. [2] with some edits to the original architecture of the mode;
For the images we added the VisionTransformer model Alexey Dosovitskiy et al. [4].
For a summary:
we reimplemented in pytorch Lightning the original paper using their code
we explored others combination of model, this is the complete list:
DGCNN+ResNet (Reference paper)
Naive Point Cloud Transformer + Vision Transformer (Transformer-based crosspoint)
Naive Point Cloud Transformer + ResNet 
DGCNN + VisionTransformer

we worked on the classification task, discarding the segmentation task due to the hardware available for training.


### DRIVE CONNECTION

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


### INSTALL DEPENDECIES

In [ ]:
!pip install pytorch_lightning --quiet
!pip install lightly --quiet
!pip install wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.6/530.6 KB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.2 MB/s eta 0:00:00


### Requirements per pointbert (forget about this 😞)


### IMPORT

In [ ]:
import argparse
import pytorch_lightning as pl
import torch
from torchvision import transforms
import numpy as np
import wandb
from lightly.loss.ntx_ent_loss import NTXentLoss
from torch.utils.data import DataLoader
#from datasets.data import ShapeNetRender, ModelNet40SVM
from drive.MyDrive.CrossPoint_Sapienza_NN_2023.util import IOStream, AverageMeter
import torch.nn as nn
import torch.nn.functional as F

#from models.dgcnn import DGCNN, ResNet, DGCNN_partseg
from torchvision.models import resnet50, resnet18
from torchvision.models import vit_l_16
#from models/pointbert/Point-BERT/models import Point_Bert

#path management
import sys

#cache pyton management
import imp
#for _init_() function to save checkpoints
import os

from sklearn.svm import SVC

### CONSTANTS
❗❗❗❗ Change the name of the experiment when you do a training with different combination of models 



In [ ]:
#COSTANTI
exp_name = 'PCT+VisionTransformer' # Name of the experiment
model_point = 'pct' # Model to use, [dgcnn, pct]
model_img = 'visiontransformer' # Model to use for images feature extraction, [resnet, visiontrasformer]
batch_size = 5 # (Size of training batch)
val_batch_size = 2 # (Size of validation batch)
test_batch_size = 2 # (Size of testing batch)
epochs = 100 # number of episode to train
start_epoch = 0  # number of episode to train
use_sgd = False # Use SGD
lr = 0.001 # learning rate (default: 0.001, 0.1 if using sgd)
momentum = 0.9 # SGD momentum (default: 0.9)
no_cuda = False # enables CUDA training
seed = 1 # random seed (default: 1)'
evalu = False # evaluate the model
num_points = 2048 # num of points to use
dropout = 0.5 # dropout rate
emb_dims = 1024 # Dimension of embeddings
k = 20 # Num of nearest neighbors to use
resume = False  # resume from checkpoint
model_path = '' # Pretrained point model path.   #/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/nomeexp/
#model_img_path = '' # Pretrained img model path.   #/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/nomeexp/
save_freq = 2 # save frequency
print_freq = 50 # print frequency
enable_wandb = True # enables wandb
spct_model_path = "/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/models/naive_pct.t7"

#use a dictionary
args = {
    'exp_name': exp_name,
    'model_point': model_point,
    'model_img': model_img,
    'batch_size': batch_size,
    'val_batch_size' : val_batch_size,
    'test_batch_size': test_batch_size,
    'epochs': epochs,
    'start_epoch': start_epoch,
    'use_sgd': use_sgd,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'seed': seed,
    'evalu': evalu,
    'num_points': num_points,
    'dropout': dropout,
    'emb_dims': emb_dims,
    'k': k,
    'resume': resume,
    'model_path': model_path,
    #'model_img_path': model_img_path,
    'save_freq': save_freq,
    'print_freq': print_freq,
    'enable_wandb': enable_wandb,
    'spct_model_path': spct_model_path
}




### FUNCTIONS (DGCNN) ⏩⏩⏩

In [ ]:
def check_checkpoint_folders(): 
  if not os.path.exists('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints'):
        os.makedirs('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints')
  if not os.path.exists('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/'+ args['exp_name']):
      os.makedirs('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/'+ args['exp_name'])
  if not os.path.exists('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/'+args['exp_name']+'/'+'models'):
      os.makedirs('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/'+args['exp_name']+'/'+'models')

def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None):
    # SPATIAL TRANSFORM
    # Application of the transformation of a point cloud in a graph considering the k neighbour 
    # Then conversion into canonical pose (from PointNet to normalize point cloud)
    # -----------------------------------

    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k)

    
    device = torch.cuda.current_device()

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points
    idx = idx.to(device)
    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    #batch_size = batch_size.to(device)
    #num_points = num_points.to(device)

    x = x.to(device)
    idx = idx.to(device)

    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    # Conversion into a canonical pose ()
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous() 
  
    return feature


### DGCNN --- CHECK THE MODEL ARCHITECTURE🔗
In this project we provide a translation in Pytoch Lightning of the DGCNN [3] model (Dynamic Graph Convolutional Neural Networks) which can be used as a feature extractor in various computer vision tasks, such as object recognition, semantic segmentation, and image classification. In this role, the DGCNN network is trained on a large dataset to learn high-level features that are representative of the objects in the images. The extracted features can be used for a variety of tasks, such as object recognition, semantic segmentation, and image classification (as in our case).



In [ ]:
class DGCNN(pl.LightningModule):
    def __init__(self, args, cls = -1) -> None:
        super(DGCNN,self).__init__()
        self.k = args['k']

        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args['emb_dims'])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),self.bn1, nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),self.bn2,nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),self.bn3,nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False), self.bn4, nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args['emb_dims'], kernel_size=1, bias=False), self.bn5, nn.LeakyReLU(negative_slope=0.2))

        if cls != -1:
            self.linear1 = nn.Linear(args['emb_dims']*2, 512, bias=False)
            self.bn6 = nn.BatchNorm1d(512)
            self.dp1 = nn.Dropout(p=args['dropout'])
            self.linear2 = nn.Linear(512, 256)
            self.bn7 = nn.BatchNorm1d(256)
            self.dp2 = nn.Dropout(p=args['dropout'])
            self.linear3 = nn.Linear(256, output_channels)
        
        self.cls = cls

        self.inv_head = nn.Sequential(
              nn.Linear(args['emb_dims'] * 2, args['emb_dims']),
              nn.BatchNorm1d(args['emb_dims']),
              nn.ReLU(inplace=True),
              nn.Linear(args['emb_dims'], 256)
          )

    def forward(self, x):
        batch_size = x.size(0)
        # SPATIAL TRANSFORMATION
        x = get_graph_feature(x, k=self.k)
        x = self.conv1(x)
        x1 = x.max(dim=-1, keepdim=False)[0]


        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x2, k=self.k)
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]

        x = torch.cat((x1, x2, x3, x4), dim=1)


        x = self.conv5(x)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)
        x = torch.cat((x1, x2), 1)

        
        feat = x
        if self.cls != -1:
            x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2)
            x = self.dp1(x)
            x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2)
            x = self.dp2(x)
            x = self.linear3(x)
        
        inv_feat = self.inv_head(feat)
        

        
        return x, inv_feat, feat 



### RESNET

In [ ]:
class ResNet(pl.LightningModule):
    def __init__(self, model, feat_dim = 2048):
        super(ResNet, self).__init__()
        self.resnet = model
        self.resnet.fc = nn.Identity()
        
        self.inv_head = nn.Sequential(
                            nn.Linear(feat_dim, 512, bias = False),
                            nn.BatchNorm1d(512),
                            nn.ReLU(inplace=True),
                            nn.Linear(512, 256, bias = False)
                            ) 
        
    def forward(self, x):
        x = self.resnet(x)
        #print("SHAPE DI X in RESNET ---->",x.shape) # (batchsize x 2048)
        x = self.inv_head(x)
        
        return x

### VISION TRANSFORMER 🎉🎉🎉
On the other side, in addition of ResNet [5] as in the paper,  we implement a Vision Transformer model as feature extractor for the rendered 2D images, however doing a comparison of the performances using either one or the other.
In a Vision Transformer, the image is first divided into a set of overlapping patches, and the features of each patch are then fed into the Transformer as individual tokens. The Transformer then processes these features using its self-attention mechanism, allowing it to capture the relationships between the patches and learn a representation of the entire image. The output of the Transformer is then processed by additional layers to make predictions for tasks such as image classification, object detection, and segmentation.


In [ ]:
import torchvision.models as models
class VisionTransformer(pl.LightningModule):
    def __init__(self, model, feat_dim = 1000):
        super(VisionTransformer, self).__init__()
        self.vision_transformer = model

        for param in self.vision_transformer.parameters():
            param.requires_grad = False

        self.inv_head = nn.Sequential(
            nn.Linear(feat_dim, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256, bias=False)
        )
        
    def forward(self, x):
        x = self.vision_transformer(x) #get output of hidden states
        #print("SHAPE DI X in ViT ---->",x.shape) # (batchsize, numclasses) -> (2,1000)
        #x = x[:, 0, :]
        x = self.inv_head(x)
        
        return x

### PointCloudTransformer (Naive) 🙈
PCT is a deep learning architecture that processes point clouds using the Transformer architecture;                 the Transformer allows the model to learn global relationships between points in the point cloud by using self-attention mechanisms.
In a Point Cloud Transformer model, each point in the point cloud is represented as a feature vector, and the Transformer processes these feature vectors to generate a compact, dense representation of the entire point cloud. This compact representation can then be used for tasks such as object classification, segmentation … 
In this project we use a naive version of this model, called “naive PCT” which differs from the original one because it typically involves directly applying the Transformer architecture to the point cloud data without any additional pre-processing or modifications. In a naive PCT model, each point in the point cloud is treated as an individual unit, and the Transformer processes these units to generate a compact representation of the entire point cloud.


In [ ]:
import math
def messingWithKeys():
  state_dict = torch.load(args['spct_model_path'])
  point_model = Pct()

  new_state_dict = {}
  not_desired_keys = ["module.cls.linear1.weight", "module.cls.linear2.weight", "module.cls.linear2.bias", "module.cls.bn1.weight", "module.cls.bn1.bias", "module.cls.bn1.running_mean", "module.cls.bn1.running_var", "module.cls.bn1.num_batches_tracked", "module.cls.bn2.weight", "module.cls.bn2.bias", "module.cls.bn2.running_mean", "module.cls.bn2.running_var", "module.cls.bn2.num_batches_tracked"]
  for k, v in state_dict.items():
      if k not in not_desired_keys:
        new_k = k.replace("module.encoder.", "")
        new_state_dict[new_k] = v

  point_model.load_state_dict(new_state_dict)
  return point_model

class Embedding(nn.Module):
    """
    Input Embedding layer which consist of 2 stacked LBR layer.
    """

    def __init__(self, in_channels=3, out_channels=128):
        super(Embedding, self).__init__()

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=1, bias=False)

        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
    
    def forward(self, x):
        """
        Input
            x: [B, in_channels, N]
        
        Output
            x: [B, out_channels, N]
        """
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x


class SA(nn.Module):
    """
    Self Attention module.
    """

    def __init__(self, channels):
        super(SA, self).__init__()

        self.da = channels // 4

        self.q_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.q_conv.weight = self.k_conv.weight
        self.v_conv = nn.Conv1d(channels, channels, 1)
        
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x):
        """
        Input
            x: [B, de, N]
        
        Output
            x: [B, de, N]
        """
        # compute query, key and value matrix
        x_q = self.q_conv(x).permute(0, 2, 1)  # [B, N, da]
        x_k = self.k_conv(x)                   # [B, da, N]        
        x_v = self.v_conv(x)                   # [B, de, N]

        # compute attention map and scale, the sorfmax
        energy = torch.bmm(x_q, x_k) / (math.sqrt(self.da))   # [B, N, N]
        attention = self.softmax(energy)                      # [B, N, N]

        # weighted sum
        x_s = torch.bmm(x_v, attention)  # [B, de, N]
        x_s = self.act(self.after_norm(self.trans_conv(x_s)))
        
        # residual
        x = x + x_s

        return x

class Pct(pl.LightningModule):
    def __init__(self,args):
        super().__init__()
        self.args = args 

        self.embedding = Embedding(3, 128)

        self.sa1 = SA(128)
        self.sa2 = SA(128)
        self.sa3 = SA(128)
        self.sa4 = SA(128)

        self.linear = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=1, bias=False),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.2)
        )

        self.inv_head = nn.Sequential(
              nn.Linear(args['emb_dims'] * 2, args['emb_dims']),
              nn.BatchNorm1d(args['emb_dims']),
              nn.ReLU(inplace=True),
              nn.Linear(args['emb_dims'], 256)
          )

       
    def forward(self, x):
        batch_size = x.size(0)

        x = self.embedding(x) #x shape->torch.Size([2bs, 3, 2048])
        
        x1 = self.sa1(x) #x1 shape->torch.Size([2bs, 128, 2048])
        x2 = self.sa2(x1) #x2 shape->torch.Size([2bs, 128, 2048])
        x3 = self.sa3(x2) #x3 shape->torch.Size([2bs, 128, 2048])
        x4 = self.sa4(x3) #x4 shape->torch.Size([2bs, 128, 2048])
        x = torch.cat([x1, x2, x3, x4], dim=1) #x shape after cat->torch.Size([10, 512, 2048])

        x = self.linear(x) #x shape after linear->torch.Size([10, 1024, 2048])
    

        # x = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1) # 2bs,1024
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1) # 2bs,1024
        x = torch.cat((x1, x2), 1) # 2bs,2048
        feat = x
        inv_feat = self.inv_head(feat) # torch.Size([2bs, 256])

        x_max = torch.max(x, dim=-1)[0]
        x_mean = torch.mean(x, dim=-1)

        return inv_feat, x_max, x_mean, feat


### ⚡️⚡️⚡️⚡️⚡️⚡️⚡️⚡️ CROSSPOINT LIGHTNING (the lightning module). ⚡️⚡️⚡️⚡️⚡️⚡️⚡️⚡️
ShapeNetRedner in dataset/data.py
method Crosspoint.png

In [ ]:
class CrosspointLightning (pl.LightningModule):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.best_acc = 0
        
        if self.args['enable_wandb'] == True:
          self.wandb_log = {}


        print("Init CrosspointLightning v4")
        
        
        # ------------
        # models
        # ------------
        if args['model_point'] == 'dgcnn':
            self.point_model = DGCNN(args) # .to(device) ?
        elif args['model_point'] == 'pct':
            #import spct from path
            #self.point_model = messingWithKeys()
            #import for train pct
            self.point_model = Pct(args)
        elif args['model_point'] == 'pointbert':
           raise Exception("@Not implemented PointBert")            
        else:
            raise Exception("Not implemented")

        if args['model_img'] == 'resnet':
            self.img_model = ResNet(resnet50(), feat_dim = 2048) 
        elif args['model_img'] == 'visiontransformer':
            self.img_model = VisionTransformer(vit_l_16(), feat_dim = 1000) 
        else:
            raise Exception("Not implemented")

        
        self.criterion = NTXentLoss(temperature = 0.1)

        if args['enable_wandb'] == True:
            wandb.watch(self.point_model)

        if args['resume']:
            # self.point_model = self.point_model.load_state_dict(torch.load(args['model_point_path']))
            # self.img_model = self.img_model.load_state_dict(torch.load(args['model_img_path']))
            model.load_state_dict(torch.load(args['model_path']))
            print("Model Loaded !!")

    def on_train_epoch_start(self):
        print("Starting epoch ->",self.current_epoch," (training)")
        self.train_losses = AverageMeter()
        self.train_imid_losses = AverageMeter()
        self.train_cmid_losses = AverageMeter()

        
        self.point_model.train()
        self.img_model.train()

    def on_train_epoch_end(self):
      if self.args['enable_wandb'] == True:
          self.wandb_log['Train Loss'] = self.train_losses.avg
          self.wandb_log['Train IMID Loss'] = self.train_imid_losses.avg
          self.wandb_log['Train CMID Loss'] = self.train_cmid_losses.avg
      print("Ending epoch ->",self.current_epoch," (training)")     
      outstr = 'Training %d, loss: %.6f' % (self.current_epoch, self.train_losses.avg)
      self.custom_validation_step(self.args)
      io.cprint(outstr)  
        
    def training_step(self, batch, batch_idx):
        (data_t1 ,data_t2), imgs = batch
        batch_size = data_t1.size()[0]

        if args['model_point'] == 'dgcnn' or args['model_point'] == 'dgcnn_seg':
            data = torch.cat((data_t1, data_t2)) #data_t1,data_t2 shape -> [bs,2048,3]
            #data shape -> [2bs,2048,3]
            data = data.transpose(2, 1).contiguous() #data shape after transpose-> [2bs,3,2048]
            _, point_feats, _ = self.point_model(data) #point_feats shape-> [2bs,256]
            point_t1_feats = point_feats[:batch_size, :] #point_t1_feats shape-> [bs,256]
            point_t2_feats = point_feats[batch_size: , :] #point_t2_feats shape-> [bs,256]

            loss_imid = self.criterion(point_t1_feats, point_t2_feats)        
            point_feats = torch.stack([point_t1_feats,point_t2_feats]).mean(dim=0) #point_feats after mean shape-> [bs,256]
        elif args['model_point'] == 'pct':
            data = torch.cat((data_t1, data_t2)) #data shape -> [2bs,2048,3]
            data = data.transpose(2, 1).contiguous() #data shape after transpose-> [2bs,3,2048]
            embbedded_x, _, _, _  = self.point_model(data) 
            point_t1_feats = embbedded_x[:batch_size, :]
            point_t2_feats = embbedded_x[batch_size: , :]
            loss_imid = self.criterion(point_t1_feats, point_t2_feats)        
            point_feats = torch.stack([point_t1_feats,point_t2_feats]).mean(dim=0)
        else:
            raise Exception("Not implemented")
        
        img_feats = self.img_model(imgs) #imgs_feats shape->[5,256]
        
        loss_cmid = self.criterion(point_feats, img_feats)

        total_loss = loss_imid + loss_cmid
        
        if self.args['enable_wandb'] == True:
          self.wandb_log['Total train loss'+args['exp_name']] = total_loss
          self.wandb_log['Imid loss'+args['exp_name']] = loss_imid
          self.wandb_log['Cmid loss'+args['exp_name']] = loss_cmid
          wandb.log(self.wandb_log)

        if batch_idx % self.args['print_freq'] == 0:
                print('Epoch (%d), Batch(%d/%d), loss: %.6f, imid loss: %.6f, cmid loss: %.6f ' % (self.current_epoch, batch_idx, self.args['train_loader_len'], total_loss, loss_imid, loss_cmid))
        
        self.train_losses.update(total_loss.item(), batch_size)
        self.train_imid_losses.update(loss_imid.item(), batch_size)
        self.train_cmid_losses.update(loss_cmid.item(), batch_size)
        
        return total_loss
              
    def forward(self, x):
        return x

    def custom_validation_step(self,args):
      print("Starting epoch ",self.current_epoch," (validation_epoch_start)")
      self.train_val_loader = DataLoader(ModelNet40SVM(partition='train', num_points=1024),num_workers=12, batch_size=5, shuffle=True)
      self.test_val_loader = DataLoader(ModelNet40SVM(partition='test', num_points=1024),num_workers=12, batch_size=2, shuffle=True)

      feats_train = []
      labels_train = []

      self.point_model.eval()

      print("Starting epoch ",self.current_epoch," (validation_step)")
      if torch.cuda.is_available():
        device = torch.device("cuda")
      else: device = torch.device('cpu')

      for i, (data, label) in enumerate(self.train_val_loader):
            labels = list(map(lambda x: x[0],label.numpy().tolist()))
            data = data.permute(0, 2, 1)
            data = data.to(device)
            
            with torch.no_grad():
                if self.args['model_point'] == 'dgcnn':
                  feats = self.point_model(data)[2]
                elif self.args['model_point'] == 'pct':
                  feats = self.point_model(data)[3]
                else:
                  raise ("Not implemented")

            feats = feats.detach().cpu().numpy()
            for feat in feats:
                feats_train.append(feat)
            labels_train += labels

      feats_train = np.array(feats_train)
      labels_train = np.array(labels_train)

      feats_test = []
      labels_test = []

      for i, (data, label) in enumerate(self.test_val_loader):
          labels = list(map(lambda x: x[0],label.numpy().tolist()))
          data = data.permute(0, 2, 1)
          data = data.to(device)

          with torch.no_grad():
              if self.args['model_point'] == 'dgcnn':
                feats = self.point_model(data)[2]
              elif self.args['model_point'] == 'pct':
                feats = self.point_model(data)[3]
              else:
                  raise ("Not implemented")

          feats = feats.detach().cpu().numpy()
          for feat in feats:
              feats_test.append(feat)
          labels_test += labels

      feats_test = np.array(feats_test)
      labels_test = np.array(labels_test)
      
      model_tl = SVC(C = 0.1, kernel ='linear')
      model_tl.fit(feats_train, labels_train)
      test_accuracy = model_tl.score(feats_test, labels_test)

      if args['enable_wandb'] == True:
        self.wandb_log['Linear Accuracy'] = test_accuracy
      
      print(f"Linear Accuracy : {test_accuracy}")
      
      if test_accuracy > self.best_acc:
          self.best_acc = test_accuracy
          print('==> NEW RECORD on accuracy on epoch ',self.current_epoch,' --- Accuracy = ',test_accuracy,' ----> SAVING Best Model...')
          file_name = 'best_model'+self.args['model_point']+'.pth'
          save_file = os.path.join('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/' + self.args['exp_name'] + '/models/', file_name.format(epoch=self.current_epoch) )
          torch.save(self.point_model.state_dict(), save_file)
          

          file_name = 'best_model'+self.args['model_img']+'.pth'
          save_img_model_file = os.path.join('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/' + self.args['exp_name'] + '/models/', 'img_model_best'+self.args['model_img']+'.pth' )
          torch.save(self.img_model.state_dict(), save_img_model_file)

      if self.current_epoch % self.args['save_freq'] == 0:
          print('==> Saving for frequency...')
          save_file = os.path.join(f'/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/{self.args["exp_name"]}/models/','ckpt_epoch_{'+str(self.current_epoch)+'}'+self.args['model_point']+'.pth'.format(epoch=self.current_epoch))
          torch.save(self.point_model.state_dict(), save_file)

      if args['enable_wandb'] == True:
        wandb.log(self.wandb_log)

      return {'val_acc': test_accuracy}


    def configure_optimizers(self):
        parameters = list(self.point_model.parameters()) + list(self.img_model.parameters())
        
        # ------------
        # optimizer
        # ------------
        if args['use_sgd']:
            print("Use SGD")
            optimizer = torch.optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'], weight_decay=1e-6)
        else:
            print("Use Adam")
            optimizer = torch.optim.Adam(parameters, lr=args['lr'], weight_decay=1e-6)
        
        # ------------
        # scheduler
        # ------------
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args['epochs'], eta_min=0, last_epoch=-1)


        return [optimizer], [lr_scheduler]

### THE TRAIN FUNCTION 🎉🎉🎉🎉
The dataset used for pre-training the model is ShapeNet which originally consists of more than 50,000 CAD models from 55 categories but due to hardware reasons we reduce the dataset to only one class that is “airplanes” .

So the model is trained on learning feature only for this class in particular but despite this we got some decent results on the final classification. 

For a given point cloud, we randomly select a 2D image out of all the rendered images(ShapeNet Rendering), which is captured from an arbitrary viewpoint.


In [ ]:
def train(args,io):

    if args['enable_wandb'] == True:
        print('Wandb Enabled')
        wandb.init(project="CrossPoint", name=args['exp_name'])

    
    # ------------
    # data
    # ------------
    transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    train_loader = DataLoader(ShapeNetRender(transform, n_imgs = 2), num_workers=12, batch_size=args['batch_size'], shuffle=True, drop_last=True)
    args['train_loader_len'] = len(train_loader)
    # device = torch.device("cuda" if args.cuda else "cpu")
     
    # ------------
    # model
    # ------------
    model = CrosspointLightning(args)
    
    # ------------
    # training
    # ------------
    parsed_args = argparse.Namespace(**args)
    
    trainer = pl.Trainer.from_argparse_args(parsed_args,max_epochs=args['epochs'],accelerator='gpu') #??? devices = 1 ???
    
    trainer.fit(model, train_loader)

### 🚀🚀🚀🚀 THE MAIN FUNCTION 🚀🚀🚀🚀

In [ ]:
check_checkpoint_folders()
torch.set_float32_matmul_precision('medium')
io = IOStream('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/checkpoints/' + args['exp_name'] + '/run.log')
io.cprint(str(args))
args['cuda'] = not args['no_cuda'] and torch.cuda.is_available()
torch.manual_seed(args['seed'])
if args['cuda']:
    io.cprint(
        'Using GPU : ' + str(torch.cuda.current_device()) + ' from ' + str(torch.cuda.device_count()) + ' devices')
    torch.cuda.manual_seed(args['seed'])
else:
    io.cprint('Using CPU')

if args['model_point'] == 'pointbert':
  sys.path.append('content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/models/Point_BERT/models')
  from drive.MyDrive.CrossPoint_Sapienza_NN_2023.models.Point_BERT.models import Point_BERT

sys.path.append('/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/datasets')
from drive.MyDrive.CrossPoint_Sapienza_NN_2023.datasets.data import ShapeNetRender, ModelNet40SVM
#sys.path.append('content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/models/Point_BERT') # forse è /models/Point_BERT/models


# ------------
# training or testing
# ------------ 
if not args['evalu']:
    train(args, io)

{'exp_name': 'PCT+VisionTransformer', 'model_point': 'pct', 'model_img': 'visiontransformer', 'batch_size': 5, 'val_batch_size': 2, 'test_batch_size': 2, 'epochs': 100, 'start_epoch': 0, 'use_sgd': False, 'lr': 0.001, 'momentum': 0.9, 'no_cuda': False, 'seed': 1, 'evalu': False, 'num_points': 2048, 'dropout': 0.5, 'emb_dims': 1024, 'k': 20, 'resume': False, 'model_path': '', 'save_freq': 2, 'print_freq': 50, 'enable_wandb': True, 'spct_model_path': '/content/drive/MyDrive/CrossPoint_Sapienza_NN_2023/models/naive_pct.t7'}
Using GPU : 0 from 1 devices
Wandb Enabled


Init CrosspointLightning v4


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.mo

Use Adam


Training: 0it [00:00, ?it/s]

...

Epoch (98), Batch(0/809), loss: 3.507367, imid loss: 1.502808, cmid loss: 2.004559 
Epoch (98), Batch(50/809), loss: 3.971581, imid loss: 1.848995, cmid loss: 2.122586 
Epoch (98), Batch(100/809), loss: 3.833859, imid loss: 1.761558, cmid loss: 2.072300 
Epoch (98), Batch(150/809), loss: 3.868470, imid loss: 1.891715, cmid loss: 1.976755 
Epoch (98), Batch(200/809), loss: 4.251555, imid loss: 2.282412, cmid loss: 1.969143 
Epoch (98), Batch(250/809), loss: 3.423265, imid loss: 1.442322, cmid loss: 1.980943 
Epoch (98), Batch(300/809), loss: 3.292347, imid loss: 1.258242, cmid loss: 2.034105 
Epoch (98), Batch(350/809), loss: 3.531145, imid loss: 1.566891, cmid loss: 1.964254 
Epoch (98), Batch(400/809), loss: 3.917294, imid loss: 1.962068, cmid loss: 1.955227 
Epoch (98), Batch(450/809), loss: 3.661491, imid loss: 1.694487, cmid loss: 1.967004 
Epoch (98), Batch(500/809), loss: 4.041264, imid loss: 2.067240, cmid loss: 1.974024 
Epoch (98), Batch(550/809), loss: 4.486339, imid loss: 2.

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch (99), Batch(0/809), loss: 3.755253, imid loss: 1.747810, cmid loss: 2.007443 
Epoch (99), Batch(50/809), loss: 4.353641, imid loss: 2.281699, cmid loss: 2.071941 
Epoch (99), Batch(100/809), loss: 3.521624, imid loss: 1.538737, cmid loss: 1.982887 
Epoch (99), Batch(150/809), loss: 3.179818, imid loss: 1.211648, cmid loss: 1.968170 
Epoch (99), Batch(200/809), loss: 3.760816, imid loss: 1.784870, cmid loss: 1.975946 
Epoch (99), Batch(250/809), loss: 4.188596, imid loss: 2.147746, cmid loss: 2.040850 
Epoch (99), Batch(300/809), loss: 3.594396, imid loss: 1.622003, cmid loss: 1.972393 
Epoch (99), Batch(350/809), loss: 4.067648, imid loss: 2.108467, cmid loss: 1.959181 
Epoch (99), Batch(400/809), loss: 3.764773, imid loss: 1.775525, cmid loss: 1.989249 
Epoch (99), Batch(450/809), loss: 3.372734, imid loss: 1.400077, cmid loss: 1.972657 
Epoch (99), Batch(500/809), loss: 4.001813, imid loss: 2.011745, cmid loss: 1.990068 
Epoch (99), Batch(550/809), loss: 4.024331, imid loss: 2.

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


Linear Accuracy : 0.8350891410048622
Training 99, loss: 3.718217


In [ ]:
if torch.cuda.is_available(): print('Flying on gpu')
else: print('Morendo su cpu')

# Now Move on eval_ssl 🏃🏼‍♂️🏃🏼‍♂️🏃🏼‍♂️